Some examples of how to convert the main Diva input and output files to geoJSON format.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import geojson
import pydiva2d

# Input files

In [2]:
datafile = '/home/ctroupin/ULg/DIVA/BlackSea4diva/data/MLD/monthly/MLD1.dat'
contourfile = '/home/ctroupin/ULg/DIVA/BlackSea4diva/contour/coast.cont.10027'
meshfile = '/home/ctroupin/ULg/DIVA/BlackSea4diva/mesh/L2/mesh.dat'
meshtopofile = '/home/ctroupin/ULg/DIVA/BlackSea4diva/mesh/L2/meshtopo.dat'
resultfile = "/home/ctroupin/ULg/DIVA/BlackSea4diva/results/detrend/MLD01.nc"
outputdir = "/home/ctroupin/Projects/Leaflet/Diva/BlackSea/"

We load the data and results using the pydiva2d module.

## Contours

In [3]:
Contour = pydiva2d.Diva2DContours()
Contour.read_from(contourfile)

2017-04-26 15:53:29,829 - pydiva2d - ERROR - File /home/ctroupin/ULg/DIVA/BlackSea4diva/contour/coast.cont.10027 does not exist


FileNotFoundError: File does not exist

## Data

In [4]:
Data = pydiva2d.Diva2DData()
Data.read_from(datafile)
Data.count_data

2017-04-26 15:53:30,142 - pydiva2d - ERROR - File /home/ctroupin/ULg/DIVA/BlackSea4diva/data/MLD/monthly/MLD1.dat does not exist


FileNotFoundError: File does not exist

## Mesh

In [5]:
Mesh = pydiva2d.Diva2DMesh()
Mesh.read_from(meshfile, meshtopofile)
Mesh.describe()

Number of nodes: 5586
Number of interfaces: 11476
Number of elements: 5870


## Field

In [8]:
Field = pydiva2d.Diva2DResults(resultfile)

# Conversion to geoJSON

## Data

We just have to loop on the data points and add the content in the correct format.

In [19]:
geojsondata = {
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "Point",
            "coordinates": [lon, lat],
            },
        "properties" : {"field": field, "weight": weight},
     } for lon, lat, field, weight in zip(Data.x, Data.y, Data.field, Data.weight)]
}

Then we can write the new file:

In [21]:
with open(os.path.join(outputdir, outputfilename), 'w') as f:
    json.dump(geojsondata, f)

## Contours

In [15]:
geojsonContour = {
    "type": "MultiPolygon", 
    "coordinates": [[[[x, y] for x, y in zip(Contour.x[cont], Contour.y[cont])]] for cont in range(0, Contour.get_contours_number)]
}

In [23]:
with open(os.path.join(outputdir, 'contour.geojson'), 'w') as f:
    json.dump(geojsonContour, f)

## Finite-element mesh

For the mesh we use the object `MultiPolygon` from the module geojson, but it's probably easy to do without it (i.e.: do it later!).

In [24]:
NN = Mesh.nelements
verts = []
for j in range(0, NN):
    verts.append([[(Mesh.xnode[Mesh.i1[j]], Mesh.ynode[Mesh.i1[j]]),
                  (Mesh.xnode[Mesh.i2[j]], Mesh.ynode[Mesh.i2[j]]),
                  (Mesh.xnode[Mesh.i3[j]], Mesh.ynode[Mesh.i3[j]]),
                  (Mesh.xnode[Mesh.i1[j]], Mesh.ynode[Mesh.i1[j]])]]
                )

In [25]:
myPolygon = geojson.MultiPolygon(verts)

We also write it to a new file:

In [26]:
with open(os.path.join(outputdir, 'BSmeshL3.geojson'), 'w') as f:
    f.write(geojson.dumps(myPolygon))

## Field

We use the contour function of `matplotlib.pyplot` to get the iso-lines.

In [29]:
fieldcontours = plt.contour(Field.x, Field.y, Field.analysis, np.arange(-60, -10, 5))
contourlist = fieldcontours.allsegs
contourvalues = fieldcontours.levels

We create the geoJSON with some loops:

In [30]:
geojsonfield = { 
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "MultiPolygon",
            "coordinates": [[ [ [lon, lat] for lon, lat in cc] for cc in contour]],
            },
        "properties" : {"field": values},
     } for contour, values in zip(contourlist, contourvalues)] 
}

and we write it to a new file:

In [32]:
with open(os.path.join(outputdir, 'BSresult.js'), 'w') as f:
    json.dump(geojsonfield, f)

Now all the input files are ready and we can use [Leaflet GeoJSON](http://leafletjs.com/examples/geojson/) to represent the information on a map.